In [1]:
from utils.preprocessing import *
from utils.pseudo_abscence import *
from utils.random_utils import *

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\libpysal\cg\alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\libpysal\cg\alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/depr

# MAIN PREPROCESSING TASK

In [16]:
torch_dataset, numpu_dataset = data_preprocess(URLs.url600s, 50, 60, boxes.South_America)

Select the dataset: 

1. data\data.csv

2. data\data_2.csv



Analyzing locations...: 100%|██████████| 15/15 [00:11<00:00,  1.29it/s]


1. Generating pseudo-absence points:

1.1. Min-Max radius analysis


Generating Min-Max radius area ...: 100%|██████████| 15/15 [04:16<00:00, 17.12s/it]


Done!
1.2. Bioclim variable analysis ...


Statistical analysis ...: 100%|██████████| 6/6 [00:00<00:00, 374.97it/s]

Done!
1.3. Picking random pseudo-absence points
Done


In [15]:
for x,y in torch_dataset:
    print(x,y)
    break
x.shape

tensor([ 0.7073,  1.5763,  1.3382,  1.2288,  1.3265,  0.3242,  1.4479, -0.3200,
         0.9427,  1.4377, -0.2729, -0.9873,  0.0354, -1.0706,  1.4798, -0.9985,
        -0.7102,  1.3135, -1.1912,  1.2494,  1.1817]) tensor(0.)


torch.Size([21])

1
2


In [12]:
numpu_dataset[0][1]

array([ 1.47291247,  2.70883614, -1.03542995, -0.36004801, -0.11173214,
       -0.43126245,  0.29833851,  0.32598112, -0.31101484, -1.58701101,
        0.86731972,  0.39370144,  0.53405093, -0.50537248,  0.72612649,
        3.61042435, -0.67285888, -3.37158128,  0.48781463, -0.76619518,
        0.70396036,  3.63270329,  0.77345909, -1.93186617,  1.30380379,
        3.5001711 , -1.20096772, -2.39030558, -0.98215281, -1.12976639,
        0.63504726,  3.3562108 , -0.43204188, -2.87387164,  0.09302576,
        1.56868769, -0.3563525 , -2.10736621, -0.00966788, -2.09605458,
        0.34664971,  1.60658391,  0.        ,  0.        ,  1.47291247,
        2.70883614, -1.03542995, -0.36004801, -0.11173214, -0.43126245,
        0.29833851,  0.32598112, -0.31101484, -1.58701101,  0.86731972,
        0.39370144,  0.53405093, -0.50537248,  0.72612649,  3.61042435,
       -0.67285888, -3.37158128,  0.48781463, -0.76619518,  0.70396036,
        3.63270329,  0.77345909, -1.93186617,  1.30380379,  3.50

# MIN-MAX RADIUS TASK

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points 
    on the Earth's surface using the Haversine formula.
    """
    R = 6371.0  # Earth radius in kilometers

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance


In [11]:
file_list = glob.glob('/data/*.csv}')
file_list


[]

In [15]:
file_list = glob.glob(os.path.join('data/', "*.csv")) + glob.glob(os.path.join('data/', "*.xlsx"))

In [16]:
file_list

['data\\data.csv']

In [14]:
glob.glob('data/*.csv')

['data\\data.csv']

In [17]:
dependent.describe().T['mean']

Longitude   -49.153361
Latitude    -14.795433
Presence      1.000000
Name: mean, dtype: float64

In [6]:
independent = tif_to_dataframe('variables/10m', bounding_box=boxes.South_America)

In [7]:
independent

NameError: name 'independent' is not defined

In [7]:
len(dependent)

NameError: name 'dependent' is not defined

# MATCHING TASK

In [3]:
dependent = pd.read_csv('data/data.csv')

In [10]:
dependent['Presence'] = 1
dependent = dependent.drop('Unnamed: 0', axis=1)

In [11]:

for idx, isolated_vector in enumerate(dependent.values):
    
    independent['distance'] = independent.apply(lambda row: haversine(isolated_vector[1], isolated_vector[0], row['Latitude'], row['Longitude']), axis=1)
    
    nearest = independent[independent['distance'] == independent['distance'].min()]

    dependent.iloc[idx,:] = (float(nearest['Longitude']), float(nearest['Latitude']), isolated_vector[2])

    break

In [12]:
dependent

,Longitude,Latitude,Presence
0,-46.666667,-10.500000,1
1,-46.595491,-10.569050,1
2,-46.594709,-10.514405,1
3,-46.569772,-10.482991,1
4,-46.616531,-10.445170,1
...,...,...,...
209,-48.947257,-8.752118,1
210,-48.983617,-8.723262,1
211,-70.256001,0.213383,1
212,-37.519376,-10.755561,1


In [ ]:
float(nearest['Latitude']), float(nearest['Longitude'])

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np

# Create a DataFrame from your dataset
data = {
    'latitude': [40.7128, 34.0522, 51.5074],  # Example latitudes
    'longitude': [-74.0060, -118.2437, -0.1278],  # Example longitudes
    # Add more latitude and longitude data here
}

df = pd.DataFrame(data)

# Define the isolated vector
isolated_vector = (37.7749, -122.4194)  # San Francisco

# Calculate distances using Haversine formula and add a new column to the DataFrame
df['distance'] = df.apply(lambda row: haversine(isolated_vector[0], isolated_vector[1], row['latitude'], row['longitude']), axis=1)

# Find the nearest vector
nearest_row = df[df['distance'] == df['distance'].min()]

# Print the nearest vector and distance
print("Nearest vector:")
print(nearest_row)

print("Distance:", nearest_row['distance'].values[0], "km")

In [11]:
import copy 
dep_copy = copy.deepcopy(dependent)

##Variables analysis    (presence, independent)

analysis = dependent.describe().loc[['mean'], ].iloc[:, 2:-1].T.reset_index(drop=True)



,mean


In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points 
    on the Earth's surface using the Haversine formula.
    """
    R = 6371.0  # Earth radius in kilometers

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

def find_nearest_vector(dataset, isolated_vector):
    min_distance = float('inf')
    nearest_vector = None

    for vector in dataset:
        distance = haversine(isolated_vector[0], isolated_vector[1], vector[0], vector[1])
        if distance < min_distance:
            min_distance = distance
            nearest_vector = vector

    return nearest_vector, min_distance

# Example usage
dataset = [
    (40.7128, -74.0060),  # New York City
    (34.0522, -118.2437),  # Los Angeles
    (51.5074, -0.1278),    # London
]

isolated_vector = (37.7749, -122.4194)  # San Francisco

nearest_vector, distance = find_nearest_vector(dataset, isolated_vector)

print("Nearest vector:", nearest_vector)
print("Distance:", distance, "km")

In [4]:
np.linalg.norm(2-3)

1.0

In [ ]:
import pandas as pd
data = {(1,2),(3,4)}
pd.DataFrame(data)

# VARIABLE TASK

In [2]:
#DEPENDENCIES
url = URLs.url600s
bounding_box = boxes.South_America
down_boundary = 70
up_boundary = 100

In [3]:
folder = from_url_tif(url)
independent = tif_to_dataframe(folder, bounding_box)
path = create_path()
dependent = import_targets(path)
presence, dependent = get_presence_dependent(independent, dependent) 
dataframe = absence_generator(presence, dependent, independent, down_boundary, up_boundary)

Select the dataset: 

1. data\data.csv

2. data\data_2.csv



Analyzing locations...: 100%|██████████| 15/15 [00:11<00:00,  1.34it/s]


1. Generating pseudo-absence points:

1.1. Min-Max radius analysis


Generating Min-Max radius area ...: 100%|██████████| 15/15 [04:28<00:00, 17.90s/it]


Done!
1.2. Bioclim variable analysis ...


Statistical analysis ...: 100%|██████████| 6/6 [00:00<00:00, 214.31it/s]

Done!
1.3. Picking random pseudo-absence points
Done


In [5]:
dataframe.columns

Index(['Longitude', 'Latitude', 'bio1', 'bio10', 'bio11', 'bio12', 'bio13',
       'bio14', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19', 'bio2', 'bio3',
       'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'Presence', 'Longitude',
       'Latitude', 'bio1', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14',
       'bio15', 'bio16', 'bio17', 'bio18', 'bio19', 'bio2', 'bio3', 'bio4',
       'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'Presence'],
      dtype='object')

In [33]:
import copy

filtered_dataframe = copy.deepcopy(complement_df)


print('1.2. Bioclim variable analysis ...')
##Variables analysis    (presence, independent)

main_bio = ['bio1', 'bio3', 'bio5', 'bio6', 'bio7', 'bio12'] ##You can try changing these values, or use all variables

analysis = presence.describe().loc[['mean', 'max'], ].loc[:, main_bio].T.reset_index()


filtered_dataframe['distance'] = 0

for variable, mean, max_value in tqdm(analysis.values, desc = 'Statistical analysis ...'):
    filtered_dataframe['distance'] = filtered_dataframe.apply(lambda row: row['distance'] + float(np.linalg.norm(mean-row[variable]))/max_value, axis = 1)

filtered_dataframe['distance'] = filtered_dataframe.apply(lambda row: row['distance']/6, axis = 1)

filtered_dataframe = filtered_dataframe.sort_values('distance', ascending = False).head(len(presence))

print('Done!')


1.2. Bioclim variable analysis ...


Statistical analysis ...: 100%|██████████| 6/6 [00:00<00:00, 461.66it/s]

Done!


In [34]:
analysis

,index,mean,max
0,bio1,24.308668,26.329626
1,bio3,69.571686,78.265152
2,bio5,33.242191,33.891251
3,bio6,14.993390,21.440331
4,bio7,18.248802,20.230751
5,bio12,1291.066650,1436.000000


In [35]:
filtered_dataframe

,Longitude,Latitude,bio1,bio10,bio11,bio12,bio13,bio14,bio15,bio16,...,bio19,bio2,bio3,bio4,bio5,bio6,bio7,bio8,bio9,distance
79,-39.166667,-3.833333,26.228344,26.970167,25.470207,813.0,209.0,1.0,112.292633,538.0,...,78.0,8.412063,77.368294,60.748989,31.735250,20.862499,10.872751,25.963291,26.186333,0.198041
78,-39.333333,-3.833333,26.390020,27.137917,25.644541,806.0,209.0,2.0,112.347702,528.0,...,79.0,8.728624,77.790031,61.058636,32.104248,20.883499,11.220749,26.093416,26.371834,0.196256
80,-39.333333,-4.000000,26.275126,27.046499,25.504833,763.0,199.0,2.0,111.695709,501.0,...,70.0,8.867084,77.083275,63.109604,32.192001,20.688749,11.503252,26.276291,26.253000,0.194742
81,-39.166667,-4.000000,26.131416,26.905750,25.348625,791.0,205.0,3.0,111.754913,517.0,...,74.0,8.734917,76.825943,63.238815,31.941999,20.572250,11.369749,25.854500,26.062584,0.191458
82,-39.333333,-4.166667,26.195208,26.987667,25.412374,731.0,192.0,2.0,110.637039,479.0,...,60.0,9.178542,76.791832,64.593094,32.353748,20.401251,11.952497,26.188042,26.176624,0.190598
77,-39.166667,-3.666667,26.340708,27.071625,25.589958,928.0,242.0,1.0,112.997025,622.0,...,87.0,8.299251,78.046326,60.123795,31.681499,21.047750,10.633749,26.074207,26.728251,0.190522
76,-39.333333,-3.666667,26.479177,27.196959,25.750916,913.0,243.0,2.0,114.970726,609.0,...,84.0,8.629271,78.488930,58.735516,32.001751,21.007500,10.994251,26.208458,26.854542,0.189224
92,-38.333333,-4.500000,26.569822,27.291458,25.845875,842.0,213.0,3.0,105.335876,530.0,...,99.0,9.340396,79.302071,57.655811,32.399250,20.621000,11.778250,26.344334,26.964167,0.188352
91,-38.500000,-4.500000,26.743126,27.474543,26.013334,836.0,214.0,3.0,105.948097,525.0,...,90.0,9.473042,79.085358,58.677620,32.719501,20.741251,11.978251,26.495125,27.127459,0.187396
96,-38.666667,-4.666667,26.739979,27.451208,26.022751,825.0,212.0,3.0,106.383629,517.0,...,197.0,9.657291,79.394035,57.602001,32.921249,20.757500,12.163750,26.436417,27.164042,0.186916


In [36]:
dependent.columns

Index(['Longitude', 'Latitude', 'bio1', 'bio10', 'bio11', 'bio12', 'bio13',
       'bio14', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19', 'bio2', 'bio3',
       'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'distance', 'Presence'],
      dtype='object')